# Model Training for Budget Forecasting

ในโน้ตบุ๊คนี้จะทำการฝึกและเปรียบเทียบโมเดลต่างๆ สำหรับการพยากรณ์กำไรของร้านค้า โดยมีขั้นตอนดังนี้:
1. การเตรียมข้อมูล
2. การสร้างและเทรนโมเดลต่างๆ
3. การประเมินและเปรียบเทียบผล
4. การวิเคราะห์ผลและบันทึกโมเดล

In [ ]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_absolute_percentage_error, mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor
from xgboost import XGBRegressor
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, GRU, Dense, Dropout, Conv1D, MaxPooling1D
from tensorflow.keras.optimizers import Adam
from prophet import Prophet
import sys
import joblib
import json
import os
from datetime import datetime

sys.path.append('..')
from src.utils import create_dataset, evaluate_predictions, plot_predictions

%matplotlib inline
plt.style.use('seaborn')
sns.set_theme(style="whitegrid")

## 1. การโหลดและตรวจสอบข้อมูล

In [ ]:
# โหลดข้อมูล
data_path = '../data/processed/Smoothed_Combined_Profit_Data_2023_2024.xlsx'
df = pd.read_excel(data_path)

print("Dataset Info:")
print(f"Number of stores: {len(df)}")
print(f"Time period: {df.columns[1:].tolist()[0]} to {df.columns[1:].tolist()[-1]}")

# แสดงข้อมูลตัวอย่าง
print("\nSample data:")
df.head()

## 2. การเตรียมข้อมูลสำหรับการเทรนโมเดล

In [ ]:
def prepare_data(store_data: pd.Series, look_back: int = 3):
    """เตรียมข้อมูลสำหรับการเทรน"""
    # สร้าง time series
    dates = pd.date_range(start='2023-01-01', periods=len(store_data), freq='M')
    ts_data = pd.Series(store_data.values, index=dates)
    
    # Scale ข้อมูล
    scaler = MinMaxScaler(feature_range=(0, 1))
    scaled_data = scaler.fit_transform(ts_data.values.reshape(-1, 1))
    
    # แบ่ง train/test
    train_size = int(len(scaled_data) * 0.8)
    train_data = scaled_data[:train_size]
    test_data = scaled_data[train_size:]
    
    # สร้าง sequences
    X_train, y_train = create_dataset(train_data, look_back)
    X_test, y_test = create_dataset(test_data, look_back)
    
    return X_train, y_train, X_test, y_test, scaler, dates

# เลือกร้านตัวอย่าง
sample_store = df.iloc[0]
store_name = sample_store['Store Name']
store_data = sample_store.iloc[1:]

# เตรียมข้อมูล
look_back = 3
X_train, y_train, X_test, y_test, scaler, dates = prepare_data(store_data, look_back)

# ปรับรูปแบบข้อมูลสำหรับโมเดลต่างๆ
X_train_3d = X_train.reshape((X_train.shape[0], X_train.shape[1], 1))
X_test_3d = X_test.reshape((X_test.shape[0], X_test.shape[1], 1))

## 3. การสร้างและเทรนโมเดล LSTM

In [ ]:
def build_lstm(look_back: int) -> Sequential:
    """สร้างโมเดล LSTM"""
    model = Sequential([
        LSTM(100, return_sequences=True, input_shape=(look_back, 1)),
        Dropout(0.2),
        LSTM(50),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
    return model

# เทรนโมเดล LSTM
lstm_model = build_lstm(look_back)
lstm_history = lstm_model.fit(
    X_train_3d, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

# Plot training history
plt.figure(figsize=(10, 6))
plt.plot(lstm_history.history['loss'], label='Training Loss')
plt.plot(lstm_history.history['val_loss'], label='Validation Loss')
plt.title('LSTM Model Loss')
plt.xlabel('Epoch')
plt.ylabel('Loss')
plt.legend()
plt.grid(True)
plt.show()

## 4. การสร้างและเทรนโมเดล GRU

In [ ]:
def build_gru(look_back: int) -> Sequential:
    """สร้างโมเดล GRU"""
    model = Sequential([
        GRU(100, return_sequences=True, input_shape=(look_back, 1)),
        Dropout(0.2),
        GRU(50),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
    return model

# เทรนโมเดล GRU
gru_model = build_gru(look_back)
gru_history = gru_model.fit(
    X_train_3d, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

## 5. การสร้างและเทรนโมเดล CNN-LSTM

In [ ]:
def build_cnn_lstm(look_back: int) -> Sequential:
    """สร้างโมเดล CNN-LSTM"""
    model = Sequential([
        Conv1D(filters=64, kernel_size=2, activation='relu', input_shape=(look_back, 1)),
        MaxPooling1D(pool_size=2),
        LSTM(50, return_sequences=True),
        Dropout(0.2),
        LSTM(25),
        Dense(1)
    ])
    model.compile(optimizer=Adam(learning_rate=0.01), loss='mean_squared_error')
    return model

# เทรนโมเดล CNN-LSTM
cnn_lstm_model = build_cnn_lstm(look_back)
cnn_lstm_history = cnn_lstm_model.fit(
    X_train_3d, y_train,
    epochs=50,
    batch_size=32,
    validation_split=0.1,
    verbose=1
)

## 6. การเทรนโมเดล Tree-based

In [ ]:
# Random Forest
rf_model = RandomForestRegressor(
    n_estimators=100,
    max_depth=10,
    min_samples_split=5,
    random_state=42
)
rf_model.fit(X_train, y_train)

# XGBoost
xgb_model = XGBRegressor(
    n_estimators=100,
    max_depth=10,
    learning_rate=0.1,
    random_state=42
)
xgb_model.fit(X_train, y_train)

## 7. การเทรนโมเดล Prophet

In [ ]:
# เตรียมข้อมูลสำหรับ Prophet
prophet_data = pd.DataFrame({
    'ds': dates[:len(y_train)],
    'y': scaler.inverse_transform(y_train.reshape(-1, 1)).flatten()
})

# เทรนโมเดล Prophet
prophet_model = Prophet(
    yearly_seasonality=True,
    weekly_seasonality=False,
    daily_seasonality=False
)
prophet_model.fit(prophet_data)

## 9. การวิเคราะห์ความผิดพลาด

In [ ]:
# Plot error distribution
plt.figure(figsize=(15, 5))
for i, (name, pred) in enumerate(models.items(), 1):
    errors = pred - y_test_orig
    plt.subplot(2, 3, i)
    sns.histplot(errors, kde=True)
    plt.title(f'{name} Error Distribution')
    plt.xlabel('Error')

plt.tight_layout()
plt.show()

# Plot error boxplot
error_data = []
for name, pred in models.items():
    errors = pred - y_test_orig
    error_data.extend([(name, e[0]) for e in errors])

error_df = pd.DataFrame(error_data, columns=['Model', 'Error'])

plt.figure(figsize=(10, 6))
sns.boxplot(x='Model', y='Error', data=error_df)
plt.xticks(rotation=45)
plt.title('การกระจายของความผิดพลาดแยกตามโมเดล')
plt.tight_layout()
plt.show()

# สรุปผลการเปรียบเทียบ
print("\nสรุปผลการเปรียบเทียบโมเดล:")
for name, metrics in results.items():
    print(f"\n{name}:")
    print(f"MAPE: {metrics['MAPE']:.2f}%")
    print(f"RMSE: {metrics['RMSE']:.2f}")
    print(f"R2 Score: {metrics['R2']:.4f}")